# 0. Initialize

## 0.1. Import Libraries

In [293]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
from datetime import datetime, timezone

import re
import string
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

from IPython import display
from IPython.display import clear_output
import matplotlib as mpl
from matplotlib import pyplot as plt

from IPython.display import clear_output

from google.colab import drive
drive.mount('./drive', force_remount=True)
from os.path import join

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Mounted at ./drive


## 0.2. DEFINE VARIABLES 

In [294]:
DATA_PATH = './drive/My Drive/CS412 Project Files/Round 3 Files/' # '<insert-your-training-data-path-here>'

ROUND = 3 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '28339'#'<insert-your-id-here>'
PROJECT_CODE = 'CS41256a26c7f3415'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [295]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf

,tweet_id,isPolitical
0,1597170281545551872,Yes
1,1431700027471192069,No
2,1566035577090281472,Yes
3,1591538690869940225,Yes
4,1583898169238167554,Yes
...,...,...
3795,1596978345560150016,Yes
3796,1582399680343789568,Yes
3797,1420407025847132171,No
3798,1589993032975544320,Yes


In [296]:
trainingTweetDf.isPolitical.value_counts()

Yes    2442
No     1353
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [297]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
trainingUserDf

,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
3795,ddionysos_s,No
3796,alpar_kaan,No
3797,nedenburdaysam,No
3798,muhibban_eco,No


In [298]:
trainingUserDf.isBot.value_counts()

No          3027
Yes          690
Not sure      78
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [299]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS41256a26c7f3415.html


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [300]:
PATH_TO_DOWNLOADED = DATA_PATH # 'D:/Users/suuser/Desktop/Sabancı/CS412/spring-2022/project/'

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data

In [301]:
'''def test(jsoninput): #test function to show how json inputs work and are parsed. Switch the "service" field between null and a value to see
    a = jsoninput['service']

    if a:
        a = a.lower()
        print("True: " + a)
    else:
        print("False: ", a)


data = '{ "name":"WandaVision", "service":"1" }'

dict_1 = json.loads(data)
test(dict_1)
'''

'def test(jsoninput): #test function to show how json inputs work and are parsed. Switch the "service" field between null and a value to see\n    a = jsoninput[\'service\']\n\n    if a:\n        a = a.lower()\n        print("True: " + a)\n    else:\n        print("False: ", a)\n\n\ndata = \'{ "name":"WandaVision", "service":"1" }\'\n\ndict_1 = json.loads(data)\ntest(dict_1)\n'

#### 1.1.1.1. Check if Retweet

In [302]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status'] #check if "retweeted_status" exists (may or may not, no guarantee). If so, return info on it
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.2. Get Tweet Text

In [303]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text'].lower()
    
    return text

#### 1.1.1.3. Get Tweet ID

In [304]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [305]:
def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])

    return num_mentions, num_hashtags

#### 1.1.1.5. Get Number of Retweets and Favorites

In [306]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.6. Get User Info

In [307]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

#### 1.1.1.7. Check If Tweet Was Made As a Reply To Another User

In [308]:
def check_if_reply(tweet_metadata_line):
    is_reply = 0
    parent_username = None

    try:
        parent_username = tweet_metadata_line['in_reply_to_screen_name']
        if parent_username:
            parent_username = parent_username.lower()
            is_reply = 1

    except KeyError:
        pass

    return is_reply, parent_username

#### 1.1.1.8. Get User Location at the Time of Tweet

In [309]:
def get_user_location(tweet_metadata_line): #no guarantee of this being simple. ex. "Meram, Konya", "Ankara"
    location = tweet_metadata_line['user']['location']

    if(location == ""):#no location info
        return False

    return location.lower()

#### 1.1.1.9. Is the User Verified?

In [310]:
def get_verified(tweet_metadata_line):
    verified = tweet_metadata_line['user']['verified']
    return verified

#### 1.1.1.10. Age of Account

In [311]:
def get_age(tweet_metadata_line):
    str_age = tweet_metadata_line['user']['created_at'] # ex. "Tue Sep 06 13:22:17 +0000 2020",
    age = datetime.strptime(str_age, '%a %b %d %H:%M:%S %z %Y') #date formats: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

    return age

#### 1.1.1.11. Get Friends and Followers

In [312]:
def get_friends_followers(tweet_metadata_line):
    friends = tweet_metadata_line['user']['friends_count']
    followers = tweet_metadata_line['user']['followers_count']

    return friends, followers

#### 1.1.1.12 Number of Tweets They Have Made

In [313]:
def get_tweet_count(tweet_metadata_line): #can be used along with age to see tweets per day ratio (how active they are)
    tweets = tweet_metadata_line['user']['statuses_count']

    return tweets

#### 1.1.1.13 Get How Many Times They Have Been Listed (tells you how influential the account is)

In [314]:
def get_listed_count(tweet_metadata_line):
    listedCount = tweet_metadata_line['user']['listed_count']
    return listedCount

#### 1.1.1.14 The Source of the Tweet (PC or Mobile)

In [315]:
def get_tweet_source(tweet_metadata_line):
    tweetSource = tweet_metadata_line['source'].lower()
    forLoc = tweetSource.find("for")
    tweetSource.replace(tweetSource[0:forLoc+4], "")

    return tweetSource

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [316]:
def check_political_ent(text):
    
    # the list below can be modified and some new names may be added (or removed)
    list_of_entities = ['06melihgokcek', 'ahmet_davutoglu', 'akaraismagiloglu', 'alibabacan', 'alimahir', 'arzuerdemdb', 'avhamzadag', 'barisatay', 
                        'bulent_arinc', 'by', 'bybekirbozdag', 'cbabdullahgul', 'celebimehmeta', 'chpmuratbakan', 'dbdevletbahceli', 'deryayanikashb', 
                        'dogu_perincek', 'drfahrettinkoca', 'durmusyillmaz', 'e_semihyalcin', 'eczozgurozel', 'ekrem_imamoglu', 'enginozkoc', 'erbakanfatih', 
                        'erkbas', 'fahrettinaltun', 'fatih_donmez', 'fethigurer', 'fuatoktay', 'ifarukaksu', 'ikalin1', 'jsarieroglu' 'kasapoglu', 'kilicdarogluk', 
                        'mahirunal', 'mansuryavas06', 'mehmedmus', 'mehmetnuriersoy', 'meral_aksener', 'mevlutcavusoglu', 'mkulunk', 'murat_kurum', 
                        'mustafa_destici', 'mustafaelitas', 'numankurtulmus', 'nureddinnebati', 'nurettincanikli', 'omerrcelik', 'rterdogan', 'seyit_torun', 
                        'suleymansoylu', 't_karamollaoglu', 'tcaytunciray', 'tuncsoyer', 'umitozdag', 'vahitkirisci', 'varank', 'vedatbilgn', 'vekilince', 
                        'yavuzagiraliog', 'yavuzyilmazd','tcmeb','herkesicinchp','akparti','iyiparti','hdpgenelmerkezi','devapartisi','tdp_turkiye','zaferpartisi',
                        'memleketimparti','gelecekpartitr','saadetpartisi','mhp_bilgi','_mhpistanbul_','btp_parti','bbpgenelmerkez','rprefahpartisi','uabakanligi',
                        'tctarim','ticaret','tcsanayi','saglikbakanligi','tcsavunma','tckulturturizm','tc_icisleri','hmbakanligi','gencliksporbak','tcenerji',
                        'tc_disisleri','csbgovtr','csgbakanligi','tcailesosyal','adalet_bakanlik']
    
    entities_in_text = [ent for ent in list_of_entities if ent.lower() in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

#### 1.1.2.2. Number of total interactions

In [317]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

#### 1.1.2.3. Amount of Numbers in Username

In [318]:
def UsernameNumberCount(displayName):
    numberCount = 0
    
    for i in displayName:
        if i.isnumeric():
            numberCount += 1

    return numberCount

In [319]:
def UsernameSpecialCharacterCount(displayName):
    specialCount = 0
    
    for i in displayName:
        if not i.isalpha():
            specialCount += 1

    return specialCount

#### 1.1.2.4. Friends to Followers ratio

In [320]:
def FriendsVFollowers(friends, followers):

  if (followers != 0):
    return friends/followers
  
  else:
    return friends/1

### 1.1.2.5. Account Activeness

In [321]:
def TweetsPerDay(tweetCount, age):
    now = datetime.now(timezone.utc)
    deltaTime = now-age

    return tweetCount/deltaTime.days

### 1.1.2.6. Collect data using the functions above and transform into a Pandas DataFrame

In [322]:
dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'num_political_entities':[],
              'total_interactions':[],
              'total_special_char_in_username':[],
              'friend_ratio':[],
              'account_activeness':[],
              'is_reply':[],
              'reply_parent':[],
              'location':[],
              'is_verified':[],
              'account_age':[],
              'friend_count':[],
              'follower_count':[],
              'tweets_count':[],
              'tweets_listed_count':[],
              'tweet_source':[]}


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        get_tweet_source
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text = get_tweet_text(line)
        num_mentions, num_hashtags = get_number_mentions_hashtags(line)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)
        is_reply, reply_parent = check_if_reply(line)
        location = get_user_location(line)
        is_verified = get_verified(line)
        account_age = get_age(line)
        friend_count, follower_count = get_friends_followers(line)
        tweets_count = get_tweet_count(line)
        tweets_listed_count = get_listed_count(line)
        tweet_source = get_tweet_source(line)#not used or placed in df

        # manually crafted data:
        num_political_entities = check_political_ent(text)
        total_num_interactions = total_interactions(retweet_count, favorite_count)
        total_special_char_in_username = UsernameSpecialCharacterCount(screen_name)
        friend_ratio = FriendsVFollowers(friend_count, follower_count)
        account_activeness = TweetsPerDay(tweets_count, account_age)

        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['num_political_entities'].append(num_political_entities)
        dfPolitical['total_interactions'].append(total_num_interactions)
        dfPolitical['total_special_char_in_username'].append(total_special_char_in_username)
        dfPolitical['friend_ratio'].append(friend_ratio)
        dfPolitical['account_activeness'].append(account_activeness)
        dfPolitical['is_reply'].append(is_reply)
        dfPolitical['reply_parent'].append(reply_parent)
        dfPolitical['location'].append(location)
        dfPolitical['is_verified'].append(is_verified)
        dfPolitical['account_age'].append(account_age)
        dfPolitical['friend_count'].append(friend_count)
        dfPolitical['follower_count'].append(follower_count)
        dfPolitical['tweets_count'].append(tweets_count)
        dfPolitical['tweets_listed_count'].append(tweets_listed_count)
        dfPolitical['tweet_source'].append(tweet_source)

In [323]:
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical.head()

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,total_special_char_in_username,friend_ratio,account_activeness,is_reply,reply_parent,location,is_verified,account_age,friend_count,follower_count,tweets_count,tweets_listed_count,tweet_source
0,1588568792984346624,0,None,"sosyal hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,1,1.897887,1.372529,0,None,ankara,False,2017-10-19 10:43:55+00:00,539,284,2638,0,"<a href=""http://twitter.com/download/android"" ..."
1,1588452263047069697,0,None,"@mahirunal gavur i̇zmir ya onlar, hani cumhuri...",1,0,0,0,595514060,mtfdan,,1,0,0,4.496183,1.709180,1,mahirunal,False,False,2012-05-31 14:08:26+00:00,589,131,6647,3,"<a href=""http://twitter.com/download/android"" ..."
2,1569589330544398336,0,None,#şehitadayıuzmçvşakadro\nsiz i̇stesenizde iste...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,0,1.966667,4.061111,0,None,False,False,2021-02-01 22:56:06+00:00,118,60,2924,0,"<a href=""http://twitter.com/download/iphone"" r..."
3,1570428119609139201,0,None,@ajans_muhbir siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,0,34.500000,4.655352,1,ajans_muhbir,False,False,2022-01-05 17:08:49+00:00,69,2,1783,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
4,1551163840368414722,0,None,engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,8,1.394958,25.975945,0,None,False,False,2022-04-07 07:58:42+00:00,166,119,7559,0,"<a href=""http://twitter.com/download/android"" ..."


### 1.1.2.7. Process Account Activeness to Fit the Data Into [0, 1] Range

In [324]:
from sklearn import preprocessing

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

dfPolitical['account_activeness'] = NormalizeData(dfPolitical['account_activeness'])

dfPolitical['account_activeness']

0        0.000082
1        0.000103
2        0.000244
3        0.000280
4        0.001561
           ...   
33528    0.000635
33529    0.000686
33530    0.000018
33531    0.019100
33532    0.000254
Name: account_activeness, Length: 33533, dtype: float64

### 1.1.2.8. Rework the Text For Our Training Data

In [325]:
dummy_array = dfPolitical

In [326]:
def remove_punctuation(text):
    no_punc = [words for words in text if words not in string.punctuation]
    word_wo_punc = "".join(no_punc)
    return word_wo_punc

dummy_array['text'] = dummy_array['text'].apply(lambda x: remove_punctuation(x))
dummy_array['text'] = dummy_array['text'].apply(lambda x: x.replace("\r", " "))
dummy_array['text'] = dummy_array['text'].apply(lambda x: x.replace("\n", " "))

dummy_array['text']

0        sosyal hizmetin temelini çocuk oluştururçocuğu...
1        mahirunal gavur i̇zmir ya onlar hani cumhuriye...
2        şehitadayıuzmçvşakadro siz i̇stesenizde isteme...
3        ajansmuhbir siz kaypak olmayıp onay vermeseydi...
4        engelli öğretmenler olarak önümüzdeki engeller...
                               ...                        
33528    gerçek kimlik taşımayan hesaplara cevap vermem...
33529    umitozdag neden suriyelilerle ilgili bu kadar ...
33530    celebimehmeta niye türkiye yüzyılıdatürkiye cu...
33531    rt muazzezeralp doan58213655 denizkonur nagiha...
33532    bybekirbozdag yasal işlemlerdn dolayı banka se...
Name: text, Length: 33533, dtype: object

In [327]:
def remove_numeric(corpus):
    output = "".join(words for words in corpus if not words.isdigit())
    return output

dummy_array['text'] = dummy_array['text'].apply(lambda x: remove_numeric(x)) 
dummy_array.head()

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,total_special_char_in_username,friend_ratio,account_activeness,is_reply,reply_parent,location,is_verified,account_age,friend_count,follower_count,tweets_count,tweets_listed_count,tweet_source
0,1588568792984346624,0,None,sosyal hizmetin temelini çocuk oluştururçocuğu...,0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,1,1.897887,0.000082,0,None,ankara,False,2017-10-19 10:43:55+00:00,539,284,2638,0,"<a href=""http://twitter.com/download/android"" ..."
1,1588452263047069697,0,None,mahirunal gavur i̇zmir ya onlar hani cumhuriye...,1,0,0,0,595514060,mtfdan,,1,0,0,4.496183,0.000103,1,mahirunal,False,False,2012-05-31 14:08:26+00:00,589,131,6647,3,"<a href=""http://twitter.com/download/android"" ..."
2,1569589330544398336,0,None,şehitadayıuzmçvşakadro siz i̇stesenizde isteme...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,0,1.966667,0.000244,0,None,False,False,2021-02-01 22:56:06+00:00,118,60,2924,0,"<a href=""http://twitter.com/download/iphone"" r..."
3,1570428119609139201,0,None,ajansmuhbir siz kaypak olmayıp onay vermeseydi...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,0,34.500000,0.000280,1,ajans_muhbir,False,False,2022-01-05 17:08:49+00:00,69,2,1783,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
4,1551163840368414722,0,None,engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,8,1.394958,0.001561,0,None,False,False,2022-04-07 07:58:42+00:00,166,119,7559,0,"<a href=""http://twitter.com/download/android"" ..."


### 1.1.2.9. Getting the Most Repeated 10000 Words to Tokenize

In [328]:
num_words = 10000
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(dummy_array['text'])

In [329]:
dummy_array_tokens = tokenizer.texts_to_sequences(dummy_array['text'])

num_tokens = [len(tokens) for tokens in dummy_array_tokens]
num_tokens = np.array(num_tokens)
num_tokens

array([5, 9, 8, ..., 6, 4, 8])

### 1.1.2.9.2. Defining a Function to Reverse Map the Tokens

In [330]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = " ".join(words) # Words will be written consecutively with space between them.
    return text

### 1.1.2.10 Appending the Token Arrays Into Our Data

In [331]:
dfPolitical['tokenized_text'] = dummy_array_tokens
dfPolitical.head()

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,total_special_char_in_username,friend_ratio,account_activeness,is_reply,reply_parent,location,is_verified,account_age,friend_count,follower_count,tweets_count,tweets_listed_count,tweet_source,tokenized_text
0,1588568792984346624,0,None,sosyal hizmetin temelini çocuk oluştururçocuğu...,0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,1,1.897887,0.000082,0,None,ankara,False,2017-10-19 10:43:55+00:00,539,284,2638,0,"<a href=""http://twitter.com/download/android"" ...","[71, 5630, 410, 6453, 5631]"
1,1588452263047069697,0,None,mahirunal gavur i̇zmir ya onlar hani cumhuriye...,1,0,0,0,595514060,mtfdan,,1,0,0,4.496183,0.000103,1,mahirunal,False,False,2012-05-31 14:08:26+00:00,589,131,6647,3,"<a href=""http://twitter.com/download/android"" ...","[284, 1610, 37, 484, 503, 9167, 130, 1870, 1610]"
2,1569589330544398336,0,None,şehitadayıuzmçvşakadro siz i̇stesenizde isteme...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,0,1.966667,0.000244,0,None,False,False,2021-02-01 22:56:06+00:00,118,60,2924,0,"<a href=""http://twitter.com/download/iphone"" r...","[7572, 52, 25, 4460, 26, 4081, 970, 544]"
3,1570428119609139201,0,None,ajansmuhbir siz kaypak olmayıp onay vermeseydi...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,0,34.500000,0.000280,1,ajans_muhbir,False,False,2022-01-05 17:08:49+00:00,69,2,1783,0,"<a href=""https://mobile.twitter.com"" rel=""nofo...","[5632, 52, 9168, 4082, 538, 37, 5, 91, 1959, 3..."
4,1551163840368414722,0,None,engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,8,1.394958,0.001561,0,None,False,False,2022-04-07 07:58:42+00:00,166,119,7559,0,"<a href=""http://twitter.com/download/android"" ...","[84, 51, 15, 1125, 2993, 4083, 84, 226, 3493, ..."


### 1.2.1. Get user metadata from user_profiles.jsons.gz

#### 1.2.1.1. Get user info metadata

In [332]:
def get_user_info_metadata(user_metadata_line):
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    
    favourites_count = user_metadata_line['favourites_count']
    geo_enabled = user_metadata_line['geo_enabled']
    verified = user_metadata_line['verified']
    statuses_count = user_metadata_line['statuses_count']
    protected = user_metadata_line['protected']
    listed_count = user_metadata_line['listed_count']
    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name,
        'user_location':user_location, 'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count,
        
        'favourites_count':favourites_count, 'geo_enabled':geo_enabled,
        'verified':verified, 'statuses_count':statuses_count, 'protected':protected, 'listed_count':listed_count
        }

    return dictionary

In [333]:
def get_age_user(tweet_metadata_line):
    str_age = tweet_metadata_line['created_at'] # ex. "Tue Sep 06 13:22:17 +0000 2020",
    age = datetime.strptime(str_age, '%a %b %d %H:%M:%S %z %Y') #date formats: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

    return age

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [334]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

#### 1.2.1.3. Get description length

In [335]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [336]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'favourites_count':[],
         'geo_enabled':[],
         'verified':[],
         'statuses_count':[],
         'protected':[],
         'listed_count':[],
         
         'description_len':[],
         'followers_to_all_ratio':[],
         'account_age':[],
         'account_activeness':[],
         'UsernameSpecialCharacterCount':[]}

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)
        account_age = get_age_user(line)

        dfBot['account_age'].append(account_age)
        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'],
                                                      dictionary['user_friends_count'])
        account_activeness = TweetsPerDay(dictionary['statuses_count'], account_age)
        specialCharacters = UsernameSpecialCharacterCount(dictionary['user_screen_name'])
        
        dfBot['description_len'].append(description_len)
        dfBot['followers_to_all_ratio'].append(followers_all_ratio)
        dfBot['account_activeness'].append(account_activeness)
        dfBot['UsernameSpecialCharacterCount'].append(specialCharacters)

In [337]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,favourites_count,geo_enabled,verified,statuses_count,protected,listed_count,description_len,followers_to_all_ratio,account_age,account_activeness,UsernameSpecialCharacterCount
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,17676,True,False,2551,False,0,7,0.260000,2021-08-27 13:07:30+00:00,4.963035,3
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,15474,True,False,42771,False,11,19,0.732260,2020-09-11 08:45:44+00:00,49.503472,0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,18220,False,False,14300,False,0,64,0.192308,2019-04-10 18:15:31+00:00,10.332370,8
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,26999,False,False,21303,False,5,65,0.325203,2016-01-29 11:01:25+00:00,8.350843,0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,2179,False,False,1629,False,0,23,0.505051,2013-12-01 18:16:41+00:00,0.487725,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,10820,False,False,2396,False,0,0,0.513453,2020-10-26 21:08:22+00:00,2.929095,1
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,36671,True,False,75178,False,279,100,0.975088,2010-02-03 18:39:01+00:00,15.870382,0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,7389,False,False,6482,False,0,28,0.451362,2009-05-01 13:56:23+00:00,1.292522,1
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,140095,True,False,121113,False,1,116,0.314431,2014-03-14 18:05:09+00:00,37.415199,0


### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [338]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

#### 1.2.2.2. Check median number of favorites

In [339]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [340]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[]
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))

        i += 1
        if i % 100 == 0:
            clear_output(wait=True)
            print(i/283, "%. Processed", i , "lines")

100.0 %. Processed 28300 lines


In [341]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets
0,594642154,0.115000,2.0,200
1,525600289,0.005025,1.0,199
2,931895965501534209,0.900000,0.0,200
3,1591543462746329088,0.185000,0.0,200
4,734801354749796352,1.000000,0.0,200
...,...,...,...,...
28310,1591370361488252928,0.800000,0.0,200
28311,1475272459616235525,0.825000,0.0,200
28312,1096753792731750401,0.051020,1.0,196
28313,1269527617687953409,0.095000,2.0,200


### 1.2.3. Merge dfBot and dfBotTweets

In [342]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)

dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,favourites_count,geo_enabled,verified,statuses_count,protected,listed_count,description_len,followers_to_all_ratio,account_age,account_activeness,UsernameSpecialCharacterCount,retweet_total_ratio,num_median_favorites,num_of_tweets
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,17676,True,False,2551,False,0,7,0.260000,2021-08-27 13:07:30+00:00,4.963035,3,0.395939,0.0,197.0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,15474,True,False,42771,False,11,19,0.732260,2020-09-11 08:45:44+00:00,49.503472,0,0.125000,0.0,200.0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,18220,False,False,14300,False,0,64,0.192308,2019-04-10 18:15:31+00:00,10.332370,8,0.910000,0.0,200.0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,26999,False,False,21303,False,5,65,0.325203,2016-01-29 11:01:25+00:00,8.350843,0,0.015306,1.0,196.0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,2179,False,False,1629,False,0,23,0.505051,2013-12-01 18:16:41+00:00,0.487725,0,0.659898,0.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,10820,False,False,2396,False,0,0,0.513453,2020-10-26 21:08:22+00:00,2.929095,1,0.015000,1.0,200.0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,36671,True,False,75178,False,279,100,0.975088,2010-02-03 18:39:01+00:00,15.870382,0,0.291457,2.0,199.0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,7389,False,False,6482,False,0,28,0.451362,2009-05-01 13:56:23+00:00,1.292522,1,0.061538,0.0,195.0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,140095,True,False,121113,False,1,116,0.314431,2014-03-14 18:05:09+00:00,37.415199,0,0.995000,0.0,200.0


In [344]:
from sklearn import preprocessing

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

dfBotAll['followers_to_all_ratio'] = NormalizeData(dfBotAll['followers_to_all_ratio'])
dfBotAll['retweet_total_ratio'] = NormalizeData(dfBotAll['retweet_total_ratio'])

# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [345]:
dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf, on='tweet_id')
dfPoliticalAll_train = dfPoliticalAll_train.sample(frac=1) #"merge" sorts the dataframe. Make sure to reshuffle it to reduce error chance
dfPoliticalAll_train.head()

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,total_special_char_in_username,friend_ratio,account_activeness,is_reply,reply_parent,location,is_verified,account_age,friend_count,follower_count,tweets_count,tweets_listed_count,tweet_source,tokenized_text,isPolitical
1045,1590028443861024768,0,None,polatsaroglu tcmeb kasıma bin öğretmen şubatt...,2,1,0,0,1430260079299895297,mehmet31732313,Görseldeki beyefendi Selim Sırrı Tarcan.,1,0,8,0.921053,0.000449,1,polatsaroglu,False,False,2021-08-24 20:06:20+00:00,280,304,3866,0,"<a href=""http://twitter.com/download/android"" ...","[3969, 62, 8598, 21, 58, 1088, 21, 4850, 596]",Yes
1180,1591058133555757056,0,None,eczozgurozel siz ekrem i̇mamoğluna soytari diy...,1,0,0,0,1623543698,hak_kerem,Yanlış üslup doğru sözün celladıdır.\nİstanbul...,1,0,1,2.145414,0.000136,1,eczozgurozel,türkiye,False,2013-07-26 18:07:48+00:00,959,447,7870,0,"<a href=""http://twitter.com/download/android"" ...","[158, 52, 148, 3838, 34, 28, 8321, 327]",Yes
3627,1588731535683112961,0,None,dfikrisaglar her evden bir oy kampanyasıi̇itif...,1,0,0,1,1334210827184984068,eminehlvacioglu,📌AK İCRAATLAR\nhttps://t.co/z1CpVN4uHH\nEvli,0,1,0,1.282239,0.004087,1,dfikrisaglar,False,False,2020-12-02 19:01:27+00:00,3321,2590,53176,0,"<a href=""http://twitter.com/download/android"" ...","[199, 24, 9650, 4, 165, 1403, 4459, 8702, 4391...",Yes
2119,1396956431010648066,0,None,annemin eli baya lezzetlidir anaannem diyo ki ...,0,0,0,6,1228672488143495172,haklisinnsulee,güzel şeyler oluyor sanırım,0,6,0,0.803279,0.000047,0,None,"buca, i̇zmir",False,2020-02-15 13:28:55+00:00,196,244,832,0,"<a href=""http://twitter.com/download/iphone"" r...","[2241, 2667, 3751, 47, 9, 28, 1766, 37]",No
4,1588452263047069697,0,None,mahirunal gavur i̇zmir ya onlar hani cumhuriye...,1,0,0,0,595514060,mtfdan,,1,0,0,4.496183,0.000103,1,mahirunal,False,False,2012-05-31 14:08:26+00:00,589,131,6647,3,"<a href=""http://twitter.com/download/android"" ...","[284, 1610, 37, 484, 503, 9167, 130, 1870, 1610]",Yes


### 2.1.2. Splitting the Data Into Training & Testing Datasets

In [346]:
from sklearn.model_selection import train_test_split

y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

target = dfPoliticalAll_train["isPolitical"].values.tolist()
data = dfPoliticalAll_train["text"].values.tolist()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.20, random_state=42)

### 2.1.3. Converting First 10000 Words Into Tokens

In [347]:
num_words = 10000
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(data)

In [348]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

print([X_train[1000]])
print(X_train_tokens[1000])

['adenium i̇nşallah amin 🙏🤲']
[5354, 608, 850, 5355]


### 2.1.3.2 Removing the Noise (Sentences With Word Count Over The Average)

In [349]:
num_tokens = [len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens = np.array(num_tokens)

max_tokens = np.mean(num_tokens) + (2*np.std(num_tokens))
max_tokens = int(max_tokens) #The maximum number of tokens a sentence may have

np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9715789473684211

### 2.1.4. Create Padding to Bring the Entire Array Into the Same Size

In [350]:
X_train_pad = pad_sequences(X_train_tokens, maxlen = max_tokens) 
X_test_pad = pad_sequences(X_test_tokens, maxlen = max_tokens)

print(X_train_pad.shape)
print(X_test_pad.shape)

(3040, 18)
(760, 18)


### 2.1.5. Creating the Model

In [351]:
embedding_size = 50
model = Sequential()
model.add(Embedding(input_dim = num_words, output_dim = embedding_size, input_length = max_tokens, name = "embedding_layer"))
model.add(GRU(units = 16, return_sequences = True))
model.add(GRU(units = 8, return_sequences = True))
model.add(GRU(units = 4))
model.add(Dense(1, activation = "sigmoid"))

In [352]:
model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 18, 50)            500000    
_________________________________________________________________
gru_9 (GRU)                  (None, 18, 16)            3216      
_________________________________________________________________
gru_10 (GRU)                 (None, 18, 8)             600       
_________________________________________________________________
gru_11 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 503,977
Trainable params: 503,977
Non-trainable params: 0
_________________________________________________________________


### 2.1.6. Fitting the Data Into Our Model

In [358]:
X_train_pad = np.array(X_train_pad)
y_train = np.array(y_train)

for i in range(y_train.size):
    if y_train[i] == "Yes":
        y_train[i] = 1
    else:
        y_train[i] = 0

y_train = y_train.astype(int)

model.fit(X_train_pad, y_train, epochs = 10, batch_size = 256)

Epoch 1/10
12/12 [==============================] - 0s 39ms/step - loss: 0.9661 - accuracy: 0.4414
Epoch 2/10
12/12 [==============================] - 0s 41ms/step - loss: 0.4186 - accuracy: 0.8444
Epoch 3/10
12/12 [==============================] - 1s 43ms/step - loss: 0.1983 - accuracy: 0.9977
Epoch 4/10
12/12 [==============================] - 0s 37ms/step - loss: 0.1662 - accuracy: 1.0000
Epoch 5/10
12/12 [==============================] - 0s 40ms/step - loss: 0.1491 - accuracy: 1.0000
Epoch 6/10
12/12 [==============================] - 0s 38ms/step - loss: 0.1360 - accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 0s 38ms/step - loss: 0.1252 - accuracy: 1.0000
Epoch 8/10
12/12 [==============================] - 0s 41ms/step - loss: 0.1160 - accuracy: 1.0000
Epoch 9/10
12/12 [==============================] - 0s 40ms/step - loss: 0.1080 - accuracy: 1.0000
Epoch 10/10
12/12 [==============================] - 0s 41ms/step - loss: 0.1011 - accuracy: 1.0000


### 2.1.7. Testing the Model By Creating Predictions

In [359]:
y_pred = model.predict(X_test_pad)
y_pred = y_pred.T[0]

In [360]:
cls_pred = np.array([1.0 if p > 0.5 else 0.0 for p in y_pred])
cls_true = np.array(y_test)

In [361]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

<ipython-input-361-0d0e23022ca0>:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  incorrect = np.where(cls_pred != cls_true)


In [362]:
len(incorrect)

1

## THE REST OF THE CODE UNTIL SECTION 2.2 IS DEPRECATED SINCE WE ARE USING A DIFFERENT MODEL

### Separate X and y values
We only use 7 features here to create a baseline model. However, **it** should PLEASE be enough to get good results.

In [ ]:
"""
X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags','is_reply','account_activeness', 'tweets_listed_count']]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)
"""

"\nX = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags','is_reply','account_activeness', 'tweets_listed_count']]\ny = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)\n"

#### 2.1.2.1 balance data by oversampling with SMOTE

In [ ]:
"""
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# transform the dataset
oversample = SMOTE(sampling_strategy=0.6)
undersample = RandomUnderSampler(sampling_strategy=0.6)

pipeline = Pipeline(steps=[('o', oversample), ('u', undersample)])
X, y = pipeline.fit_resample(X, y)
#X, y = oversample.fit_resample(X, y)
"""

"\nfrom imblearn.over_sampling import SMOTE\nfrom imblearn.under_sampling import RandomUnderSampler\nfrom imblearn.pipeline import Pipeline\n\n# transform the dataset\noversample = SMOTE(sampling_strategy=0.6)\nundersample = RandomUnderSampler(sampling_strategy=0.6)\n\npipeline = Pipeline(steps=[('o', oversample), ('u', undersample)])\nX, y = pipeline.fit_resample(X, y)\n#X, y = oversample.fit_resample(X, y)\n"

### Train - validation split

In [ ]:
"""from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)"""

### Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [ ]:
"""from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_political = MLPClassifier()

# fit your model
cross_v = 10

max_depth = [550]
param_grid = {'max_iter': max_depth, 'solver': ['adam'], 'activation': ['tanh'], 'hidden_layer_sizes': [(50,100,50)]}

clf_max_depth = MLPClassifier()

grid = GridSearchCV(clf_max_depth,
                    param_grid, 
                    cv=cross_v,
                    scoring='precision',
                    return_train_score=False, 
                    verbose=1, 
                    refit=True)

dtc_political = grid.fit(X_train.values, y_train)

# make predictions
preds = dtc_political.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score)"""

Fitting 10 folds for each of 1 candidates, totalling 10 fits
MSE: 0.27365728900255754 
 Accuracy Score: 0.7263427109974424


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [ ]:
"""dtc_political.best_params_"""

{'activation': 'tanh',
 'hidden_layer_sizes': (50, 100, 50),
 'max_iter': 550,
 'solver': 'adam'}

In [ ]:
"""MLPClassifier().get_params()"""

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [363]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [364]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,favourites_count,geo_enabled,verified,statuses_count,protected,listed_count,description_len,followers_to_all_ratio,account_age,account_activeness,UsernameSpecialCharacterCount,retweet_total_ratio,num_median_favorites,num_of_tweets,screen_name,isBot
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,17676,True,False,2551,False,0,7,0.260000,2021-08-27 13:07:30+00:00,4.963035,3,0.395939,0.0,197.0,nasreenakhan006,Yes
1,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,17676,True,False,2551,False,0,7,0.260000,2021-08-27 13:07:30+00:00,4.963035,3,0.395939,0.0,197.0,nasreenakhan006,Yes
2,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,17676,True,False,2551,False,0,7,0.260000,2021-08-27 13:07:30+00:00,4.963035,3,0.395939,0.0,197.0,nasreenakhan006,No
3,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,17676,True,False,2551,False,0,7,0.260000,2021-08-27 13:07:30+00:00,4.963035,3,0.395939,0.0,197.0,nasreenakhan006,No
4,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,15474,True,False,42771,False,11,19,0.732260,2020-09-11 08:45:44+00:00,49.503472,0,0.125000,0.0,200.0,scorpiehoez,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,1286770207134973954,Hamide Arabacı,anka6054,,,151,61,5347,False,False,5288,False,0,0,0.712264,2020-07-24 21:08:34+00:00,5.798246,4,0.000000,1.0,200.0,anka6054,No
3796,1598032338323214338,atamabekleyenbahceci,atamabekleyenzz,,,173,367,196,False,False,311,False,0,0,0.320370,2022-11-30 19:13:03+00:00,5.759259,0,0.580000,0.0,200.0,atamabekleyenzz,No
3797,760235343966863360,Emrah İNCİ,memrahinci,Istanbul - Bayburt,Researcher | Middle East | Political Science |...,5863,5905,725,True,False,1029,False,0,71,0.498215,2016-08-01 22:06:45+00:00,0.435095,0,0.040000,36.0,200.0,memrahinci,No
3798,1553973684100124672,Murat Kkk,muratkkk18,,Normal sıradan bir insanım,1,10,38,False,False,18,False,0,26,0.090909,2022-08-01 05:19:56+00:00,0.102857,2,0.769231,0.0,13.0,muratkkk18,No


In [365]:
trainingUserDf.isBot.value_counts()

No          3027
Yes          690
Not sure      78
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [366]:
X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites']]
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

### 2.2.3. Train-test split

In [367]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.2.4. Train the model

In [368]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_bot = RandomForestClassifier()

# fit your model
cross_v = 10

max_depth = [5]
param_grid = {'max_depth': max_depth, 'n_estimators': [14]}

clf_max_depth = RandomForestClassifier()

grid = GridSearchCV(clf_max_depth,
                    param_grid, 
                    cv=cross_v,
                    scoring='precision',
                    return_train_score=False, 
                    verbose=1, 
                    refit=True)

dtc_bot = grid.fit(X_train.values, y_train)

# make predictions
preds = dtc_bot.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n")

Fitting 10 folds for each of 1 candidates, totalling 10 fits
MSE: 0.27365728900255754 
 Accuracy Score: 0.8407894736842105 



/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [369]:
dtc_bot.best_params_

{'max_depth': 5, 'n_estimators': 14}

# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [370]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('{}evaluation-round{}-tweet.csv'.format(DATA_PATH,ROUND), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

#fix text of the new df
def remove_punctuation(text):
    no_punc = [words for words in text if words not in string.punctuation]
    word_wo_punc = "".join(no_punc)
    return word_wo_punc
def remove_numeric(corpus):
    output = "".join(words for words in corpus if not words.isdigit())
    return output
dfPolitical_test['text'] = dfPolitical_test['text'].apply(lambda x: remove_punctuation(x))
dfPolitical_test['text'] = dfPolitical_test['text'].apply(lambda x: x.replace("\r", " "))
dfPolitical_test['text'] = dfPolitical_test['text'].apply(lambda x: x.replace("\n", " "))
dfPolitical_test['text'] = dfPolitical_test['text'].apply(lambda x: remove_numeric(x))

#process the text of dfPolitical_test
num_words = 10000
tokenizer_final = Tokenizer(num_words = num_words)
tokenizer_final.fit_on_texts(dfPolitical_test["text"].values.tolist())

X_tokens = tokenizer_final.texts_to_sequences(dfPolitical_test["text"].values.tolist())

num_tokens = [len(tokens) for tokens in X_tokens]
num_tokens = np.array(num_tokens)

X_pad = pad_sequences(X_tokens, maxlen = np.max(num_tokens))

#make prediction
final_pred = model.predict(X_pad)
final_pred = final_pred.T[0]
predictions_political = np.array([1.0 if n > 0.5 else 0.0 for n in final_pred])

# define X as we did above in section (2.x.2. Separate X and y values)
# X = dfPolitical_test[['tokenized_text']]

# make predictions based on these variables
# predictions_political = model.predict(X)

### This part is important! We expect you to return your predictions in the following format:

In [371]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1434787703783051264': 0.0,
 '1367571642604544000': 0.0,
 '1589993032975544320': 0.0,
 '1565312596135354373': 0.0,
 '1579558096833511424': 0.0,
 '1439547067337256967': 0.0,
 '1559963768372740098': 0.0,
 '1562853131251118081': 0.0,
 '1586021183958704128': 0.0,
 '1585766233491886081': 0.0,
 '1427746815420604417': 0.0,
 '1352635736537882629': 0.0,
 '1415032260571680768': 0.0,
 '1548636597628899328': 0.0,
 '1564926450096013313': 0.0,
 '1585634359612420101': 0.0,
 '1597138789108895744': 0.0,
 '1391681495622995971': 0.0,
 '1389951943343316995': 0.0,
 '1452348722810138646': 0.0,
 '1595829502021623812': 0.0,
 '1413108476348354562': 0.0,
 '1579408398894137344': 0.0,
 '1570758749606019073': 0.0,
 '1366091745772077058': 0.0,
 '1588936302086737925': 0.0,
 '1584922292127256577': 0.0,
 '1586279180983042050': 0.0,
 '1434963357023473671': 0.0,
 '1399687111234756612': 0.0,
 '1363226564683522051': 0.0,
 '1596978345560150016': 0.0,
 '1593686506622914560': 0.0,
 '1350478535954198530': 0.0,
 '157227522855

## 3.2. Predictions for Users (Bot or Not)

In [372]:
evaluationUserDf = pd.read_csv('{}evaluation-round{}-user.csv'.format(DATA_PATH,ROUND), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites']]

# make predictions based on these variables
predictions_bot = dtc_bot.predict(X)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [373]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'biologselim': 0.0,
 'omerakdag34': 0.0,
 'bilgin21604923': 0.0,
 '_sydneycarton_': 0.0,
 'denizlihabercom': 0.0,
 'burakerbaychp': 0.0,
 'mvnez': 0.0,
 'qara118': 0.0,
 'nabiyonyevrum': 0.0,
 'farukhalit2': 0.0,
 'harlunoshi': 0.0,
 'heritagepaix': 0.0,
 'nuranwolf': 0.0,
 'politikgundem': 0.0,
 'isakethudax': 0.0,
 'enveraysevera': 0.0,
 'ilaydejaneiro': 0.0,
 '1905anason': 0.0,
 'eraydurgut03': 0.0,
 'dasiskein': 0.0,
 'ercan_bas29': 0.0,
 'mett_1907': 0.0,
 'ondemir066': 0.0,
 'semihyeteer': 0.0,
 'haberinyokcokk': 0.0,
 'meleky_ozaydin': 0.0,
 'mehmetaltay64': 0.0,
 'nurtencam2': 0.0,
 'bilobi4': 0.0,
 'yorumsuzadam87': 0.0,
 'twitsildiren': 0.0,
 '21gramlife1': 0.0,
 'cakan0_': 0.0,
 'oguzksalici': 0.0,
 'canpixlr': 0.0,
 'ozlemesor': 0.0,
 'emre_caliskann': 0.0,
 'oguzhanonel': 0.0,
 'mintchocolatemx': 0.0,
 'mehmet07454846': 0.0,
 'begumkarabeyx': 0.0,
 'imamgibiimam': 0.0,
 'durdane52': 0.0,
 'rk_ozanyali': 0.0,
 'suaterce': 0.0,
 'ibrahimcalsskan': 0.0,
 'goksubayy': 0.0,
 '

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [374]:
# Explain your approach

data_explanations = '''
Not much to say about how I handled the data, I mostly utilized the given codes and fixed a couple of 
small issues relating to them like specifying the 'PATH_DATA' and the formats regarding the '.csv' files.
I also added all of our project members' annotation data to the dataset to get better results.
'''

feature_explanations = '''
We have added two features to the user dataset: 'account_activeness' and 'friend_ratio'. These features are
important because political accounts are generally more active in social media whether they retweet or publish.
If an account is more active than others, there is a chance that the account could be related to someone political.
The friend ratio feature is important too because in Twitter, political accounts usually have more followers than they
follow, so if an account's ratio is lower than an average account, chances are that account is someone important or
related to an organization, sometimes political and sometimes not.

We have also added the 'is_reply' feature to the tweets dataset to see if the user is replying to another user.
Users may be messaging in reply to another figure and trolling or making political comments about them was the reasoning for this.
'''

model_explanations = '''
I have experimented with various machine learning classifiers like GaussianNB, KNearestNeigbors, LogicalRegression,
XGBoostClassifier and many more. I noticed that RandomForestClassifier gives better results for our Tweet dataset and I tried to
optimize the results by changing different parameters of the classifier, eventually deciding on the final values of the parameters.
I also noticed that for the users dataset, GaussianNB is a better alternative than RandomForest, but I unfortunately couldn't get it
to work in time, so I used RandomForestClassifier for the users dataset as well.
'''

additional_explanations = '''
While testing with different classifiers, I used the '.best_params_' method for various models to find out the most optimized values
for the parameters of the classifiers. By comparing different classifiers' most optimized results with this method, I have decided on
using RandomForestClassifier.
'''


In [375]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('{}predictions-{}_round{}.json'.format(DATA_PATH, STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [376]:
# Test your submission file

submission = json.load(open('{}predictions-{}_round{}.json'.format(DATA_PATH, STUDENT_ID, ROUND), 'r'))
submission

{'round': 3,
 'student_id': '28339',
 'user_predictions': {'biologselim': 0.0,
  'omerakdag34': 0.0,
  'bilgin21604923': 0.0,
  '_sydneycarton_': 0.0,
  'denizlihabercom': 0.0,
  'burakerbaychp': 0.0,
  'mvnez': 0.0,
  'qara118': 0.0,
  'nabiyonyevrum': 0.0,
  'farukhalit2': 0.0,
  'harlunoshi': 0.0,
  'heritagepaix': 0.0,
  'nuranwolf': 0.0,
  'politikgundem': 0.0,
  'isakethudax': 0.0,
  'enveraysevera': 0.0,
  'ilaydejaneiro': 0.0,
  '1905anason': 0.0,
  'eraydurgut03': 0.0,
  'dasiskein': 0.0,
  'ercan_bas29': 0.0,
  'mett_1907': 0.0,
  'ondemir066': 0.0,
  'semihyeteer': 0.0,
  'haberinyokcokk': 0.0,
  'meleky_ozaydin': 0.0,
  'mehmetaltay64': 0.0,
  'nurtencam2': 0.0,
  'bilobi4': 0.0,
  'yorumsuzadam87': 0.0,
  'twitsildiren': 0.0,
  '21gramlife1': 0.0,
  'cakan0_': 0.0,
  'oguzksalici': 0.0,
  'canpixlr': 0.0,
  'ozlemesor': 0.0,
  'emre_caliskann': 0.0,
  'oguzhanonel': 0.0,
  'mintchocolatemx': 0.0,
  'mehmet07454846': 0.0,
  'begumkarabeyx': 0.0,
  'imamgibiimam': 0.0,
  'du